<a href="https://colab.research.google.com/github/AUT-Student/CN-Project/blob/main/CN_Project_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
import torch

In [6]:
torch.__version__

'1.13.1+cu116'

In [7]:
!pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 93.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 KB 28.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 1.1 MB/s eta 0:00:00
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773302 sha256=e31c353cc0e0a437dff0da8b66b59ad522f7167d11f3f1363dceb32f8992a8d2
  Stored in directory: /root/.cache/pip/wheels/59/a3/20/198928106d3169865ae73afcbd3d3d1796cf6b429b55c65378
Successfully built 

In [ ]:
!pip install torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-1.13.0+cu116.html

In [4]:
import torch_geometric

# Dataset

## Create

In [5]:
from torch_geometric.datasets.dblp import DBLP

In [7]:
dblp = DBLP("/content")

Extracting /content/raw/DBLP_processed.zip
Processing...
Done!


## Overview

In [9]:
dblp.data

HeteroData(
  author={
    x=[4057, 334],
    y=[4057],
    train_mask=[4057],
    val_mask=[4057],
    test_mask=[4057]
  },
  paper={ x=[14328, 4231] },
  term={ x=[7723, 50] },
  conference={ num_nodes=20 },
  (author, to, paper)={ edge_index=[2, 19645] },
  (paper, to, author)={ edge_index=[2, 19645] },
  (paper, to, term)={ edge_index=[2, 85810] },
  (paper, to, conference)={ edge_index=[2, 14328] },
  (term, to, paper)={ edge_index=[2, 85810] },
  (conference, to, paper)={ edge_index=[2, 14328] }
)

In [13]:
dblp.data['author']

{'x': tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'y': tensor([2, 2, 3,  ..., 0, 0, 0]), 'train_mask': tensor([False, False, False,  ..., False, False, False]), 'val_mask': tensor([False, False,  True,  ..., False, False, False]), 'test_mask': tensor([ True,  True, False,  ...,  True,  True,  True])}

In [28]:
set(dblp.data['author']["y"].numpy())

{0, 1, 2, 3}

In [14]:
dblp.data['paper']

{'x': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}

In [15]:
dblp.data['term']

{'x': tensor([[-0.6924, -0.4659,  1.1540,  ...,  0.9178,  0.1995, -0.6360],
        [ 1.2031, -0.4003,  0.0740,  ...,  1.3262, -0.3325,  0.8198],
        [ 0.3748,  0.5731,  0.4802,  ...,  1.1522,  0.6010, -0.4309],
        ...,
        [ 0.4180,  0.2497, -0.4124,  ..., -0.1841, -0.1151, -0.7858],
        [ 0.1724, -0.2723, -1.3368,  ..., -0.0881,  0.0225,  0.1166],
        [ 0.2197,  0.0253,  0.1220,  ...,  0.0871, -0.5351, -0.4949]])}

## Old

Source: https://www.aminer.org/citation

In [2]:
!gdown https://lfs.aminer.cn/lab-datasets/citation/DBLP-citation-network-Oct-19.tar.gz

Downloading...
From: https://lfs.aminer.cn/lab-datasets/citation/DBLP-citation-network-Oct-19.tar.gz
To: /content/DBLP-citation-network-Oct-19.tar.gz
100% 202M/202M [00:23<00:00, 8.57MB/s]


In [9]:
!tar xf /content/DBLP-citation-network-Oct-19.tar.gz -C /content/

\#* --- paperTitle

\#@ --- Authors

\#t ---- Year

\#c  --- publication venue

\#index 00---- index id of this paper

\#% ---- the id of references of this paper (there are multiple lines, with each indicating a reference)

\#! --- Abstract

In [23]:
index = None
dataset = []
with open("/content/DBLPOnlyCitationOct19.txt") as file:

  for i, line in enumerate(file.readlines()[1:]):

    line = line.strip()

    if line[:2] == "#i":
      index = int(line[6:])

    elif line[:2] == "#%":
      reference = int(line[2:])
      dataset.append({"article": index, "reference": reference})

dataset = pd.DataFrame(dataset)
dataset = dataset.groupby("article")["reference"].apply(np.array)

In [29]:
len(dataset)

2327450

# GCN